# Cài đặt Sigopt
Sử dụng câu lệnh bên dưới để cài đặt

In [1]:
!pip install sigopt

  Using cached sigopt-8.4.0-py2.py3-none-any.whl (181 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached Pint-0.16.1-py2.py3-none-any.whl (205 kB)
  Using cached kubernetes-12.0.1-py2.py3-none-any.whl (1.7 MB)
  Using cached pypng-0.0.21-py3-none-any.whl (48 kB)
  Using cached backoff-1.11.1-py2.py3-none-any.whl (13 kB)
  Using cached docker-4.4.4-py2.py3-none-any.whl (147 kB)
  Using cached boto3-1.16.34-py2.py3-none-any.whl (129 kB)
  Using cached botocore-1.19.63-py2.py3-none-any.whl (7.2 MB)
  Using cached s3transfer-0.3.7-py2.py3-none-any.whl (73 kB)

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Ryor\\anaconda3\\Lib\\site-packages\\win32\\_win32sysloader.pyd'
Consider using the `--user` option or check the permissions.




  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.32
    Uninstalling botocore-1.24.32:
      Successfully uninstalled botocore-1.24.32
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.0
    Uninstalling s3transfer-0.5.0:
      Successfully uninstalled s3transfer-0.5.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: pywin32
    Found existing installation: pywin32 302
    Uninstalling pywin32-302:
      Successfully uninstalled pywin32-302
  Rolling back uninstall of pywin32
  Moving to c:\users\ryor\anaconda3\lib\site-packages\__pycache__\pythoncom.cpython-39.pyc
   from C:\Use

# Lấy API_TOKEN 
Đầu tiên chúng ta cần đăng nhập vầo Sigopt và chọn vào phần API Tokens để lấy API Token có mã như sao : JIGCTDCWFICIUSWUHDJIGFUUKSTRLWMCKRJXBDIDQILQTVFW

In [2]:
import sigopt
%load_ext sigopt
%sigopt config

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


SigOpt API token (find at https://app.sigopt.com/tokens/info): ··········
Log Collection
	This will capture and upload the standard output and standard error of your
	Runs from the CLI and notebook cells so that you can view them on the SigOpt dashboard.
Enable log collection [y/N]: y
Notebook Cell Tracking
	This will record and upload the content of your notebook cells so that you can view them
	on the SigOpt dashboard.
Enable cell tracking [y/N]: y


#Import thư viện
Thêm thư viện để tải dữ liệu

In [3]:
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import datasets
import numpy
import sigopt
import time

# Tải tập dữ liệu
Tải tập dữ liệu sklearn và các tính năng tỷ lệ về giá trị trung bình bằng 0, phương sai đơn vị.

In [4]:
DATASET_NAME = "Sklearn Wine"
FEATURE_ENG_PIPELINE_NAME = "Sklearn Standard Scalar"
PREDICTION_TYPE = "Multiclass"
DATASET_SRC = "sklearn.datasets"

def get_data():

  """
  Load sklearn wine dataset, and scale features to be zero mean, unit variance.
  One hot encode labels (3 classes), to be used by sklearn OneVsRestClassifier.
  """

  data = datasets.load_wine()
  X = data["data"]
  y = data["target"]

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  enc = OneHotEncoder()
  Y = enc.fit_transform(y[:, numpy.newaxis]).toarray()

  return (X_scaled, Y)

# Bây giờ chúng ta tạo hàm chức năng cho mô hình
eval_xgboost_model khởi tạo một bộ phân loại xgboost cho mỗi lớp trong tập dữ liệu 3 lớp của chúng ta và đánh giá bộ phân loại.

number_of_cross_val_folds trước khi báo cáo điểm trung bình và thời gian  để khởi tạo và đào tạo các mô hình.

In [5]:
#max_depth: Độ sâu tối đa cây quyết định 
#learning_rate: Thời gian học sau mỗi bước tăng cường
#min_split_loss:Giảm tổn thất tối thiểu cần thiết để thực hiện một phân vùng tiếp theo trên một nút của cây quết định.
MODEL_NAME = "OneVsRestClassifier(XGBoostClassifier)"

def evaluate_xgboost_model(X, y,
                           number_of_cross_val_folds=5,
                           max_depth=6,
                           learning_rate=0.3,
                           min_split_loss=0):
    t0 = time.time()
    classifier = OneVsRestClassifier(XGBClassifier(
        objective = "binary:logistic",
        max_depth =    max_depth,
        learning_rate = learning_rate,
        min_split_loss = min_split_loss,
        use_label_encoder=False,
        verbosity = 0
    ))
    cv_accuracies = cross_val_score(classifier, X, y, cv=number_of_cross_val_folds)
    tf = time.time()
    training_and_validation_time = (tf-t0)
    return numpy.mean(cv_accuracies), training_and_validation_time

# Hàm thứ hai run_and_track_in_sigopt sử dụng các phương pháp SigOpt để ghi nhật ký và theo dõi thông tin mô hình chính bao gồm:

1.  Loại mô hình được sử dụng (sigopt.log_model)
2.  Tên của tập dữ liệu (sigopt.log_dataset)
3.  Các siêu tham số được sử dụng để xây dựng mô hình (sigopt.params. [PARAMETER_NAME])
4.  Các thuộc tính khác nhau có liên quan đến mô hình (sigopt.log_metadata)
5.  Số liệu đầu ra của mô hình (sigopt.log_metric).

In [6]:
def run_and_track_in_sigopt():

    (features, labels) = get_data()

    sigopt.log_dataset(DATASET_NAME)
    sigopt.log_metadata(key="Dataset Source", value=DATASET_SRC)
    sigopt.log_metadata(key="Feature Eng Pipeline Name", value=FEATURE_ENG_PIPELINE_NAME)
    sigopt.log_metadata(key="Dataset Rows", value=features.shape[0]) 
    sigopt.log_metadata(key="Dataset Columns", value=features.shape[1])
    sigopt.log_metadata(key="Execution Environment", value="Colab Notebook")
    sigopt.log_model(MODEL_NAME)

    sigopt.params.setdefault("max_depth", numpy.random.randint(low=3, high=15, dtype=int))
    sigopt.params.setdefault("learning_rate", numpy.random.random(size=1)[0])
    sigopt.params.setdefault("min_split_loss", numpy.random.random(size=1)[0]*10)

    args = dict(X=features,
                y=labels,
                max_depth=sigopt.params.max_depth,
                learning_rate=sigopt.params.learning_rate,
                min_split_loss=sigopt.params.min_split_loss)

    mean_accuracy, training_and_validation_time = evaluate_xgboost_model(**args)

    sigopt.log_metric(name='accuracy', value=mean_accuracy)
    sigopt.log_metric(name='training and validation time (s)', value=training_and_validation_time)

Với lệnh %%experiment bên dưới,chúng ta cấu hình %%experiment bằng cách đặt tên cho nó, xác định độ chính xác làm chỉ số để tối đa hóa và cuối cùng đặt không gian siêu tham số  bằng cách cho SigOpt chạy các giá trị trong ranh giới đã đặt. Sau đó công cụ tối ưu hóa của SigOpt trả về các giá trị cho độ sâu tối đa từ 3 và 12 và tỷ lệ học tập là 0 và 1. Cuối cùng, xác định thời gian chúng ta sẽ đào tạo mô hình của mình. Cuối cùng chúng ta sẽ huấn luyện mô hình 4 lần, tương ứng với 4 lần chạy SigOpt.

In [9]:
%%experiment
{
    'name': 'XGBoost Optimization',
    'metrics': [
        {
            'name': 'accuracy',
            'strategy': 'optimize',
            'objective': 'maximize',
        }
    ],
    'parameters': [
        {
            'name': 'max_depth',
            'type': 'int',
            'bounds': {'min': 3, 'max': 12}
        },
        {
            'name': 'learning_rate',
            'type': 'double',
            'bounds': {'min': 0.0, 'max': 1.0}
        }
    ],
    'budget': 4
}

Experiment created, view it on the SigOpt dashboard at https://app.sigopt.com/experiment/522728


#Chạy tối ưu
Chạy tối ưu hóa  bằng cách sử dụng lệnh  %% optimize. SigOpt sẽ chọn cấu hình thử nghiệm tự động và thuận tiện xuất các liên kết trong thiết bị đầu cuối tới ứng dụng chạy trên web hiện tại .

In [10]:
%%optimize teamdemo
run_and_track_in_sigopt()

Run started, view it on the SigOpt dashboard at https://app.sigopt.com/run/316557
Run finished, view it on the SigOpt dashboard at https://app.sigopt.com/run/316557
Run started, view it on the SigOpt dashboard at https://app.sigopt.com/run/316558
Run finished, view it on the SigOpt dashboard at https://app.sigopt.com/run/316558
Run started, view it on the SigOpt dashboard at https://app.sigopt.com/run/316559
Run finished, view it on the SigOpt dashboard at https://app.sigopt.com/run/316559
Run started, view it on the SigOpt dashboard at https://app.sigopt.com/run/316560
Run finished, view it on the SigOpt dashboard at https://app.sigopt.com/run/316560
